In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import xlrd

In [2]:
df = pd.read_csv('dfLimpio.csv')
df

,ID,FechaIngreso,PacienteFechaNacimiento,PacienteAnos,PacienteSexo,DiasInternacion,CondicionAlCierre,NotaFechaHora,ArqutipoNombre,NotaArquetipoValorIngresado
0,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,Frecuencia Cardiaca,78.0
1,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,Frecuencia respiratoria,18.0
2,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,Presion Arterial Maxima,120.0
3,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,Presion Arterial Minima,80.0
4,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,Temperatura,36.3
...,...,...,...,...,...,...,...,...,...,...
609166,387442,2022-11-05,1976-09-05,46,MASCULINO,1,ALTA MEDICA,2022-11-05 16:00:00,Frecuencia respiratoria,19.0
609167,387442,2022-11-05,1976-09-05,46,MASCULINO,1,ALTA MEDICA,2022-11-05 16:00:00,Presion Arterial Maxima,120.0
609168,387442,2022-11-05,1976-09-05,46,MASCULINO,1,ALTA MEDICA,2022-11-05 16:00:00,Presion Arterial Minima,80.0
609169,387442,2022-11-05,1976-09-05,46,MASCULINO,1,ALTA MEDICA,2022-11-05 16:00:00,Temperatura,36.4


**Información de las columnas que contiene el Data Set:**
- ID: Número para identficar al paciente (en forma anónima)	
- FechaIngreso: Fecha y hora en el que el paciente es ingresado a Terapia Intensiva	(UCI)
- PacienteFechaNacimiento: Fecha de nacimiento del paciente. Contiene hora pero sin registrar, toda puesta a cero (dato irrelevante)
- PacienteAnos: Edad del paciente
- PacienteSexo: sexo del paciente
- DiasInternacion: cantidad de días que el paciente estuvo en Terapia Intensiva	
- CondicionAlCierre: Cuál fue la situación del paciente al dejar la Terapia Intensiva: alta, defunción, etc.	
- NotaFechaHora: Día y hora en la que se le hizo la medición de "X" signo vital al paciente
- ArqutipoNombre: signo vital que se midió del paciente	
- NotaArquetipoValorIngresado: Resultado numérico de la medición

# Preparación de datos para la creación del nuevo DS 

In [3]:
df['ArqutipoNombre'].value_counts()

Frecuencia Cardiaca        93502
Frecuencia respiratoria    93498
Temperatura                91818
Presion Arterial Maxima    91034
Presion Arterial Minima    90860
Saturacion O2              88242
Presion Arterial Media     60217
Name: ArqutipoNombre, dtype: int64

In [4]:
df.dtypes

ID                               int64
FechaIngreso                    object
PacienteFechaNacimiento         object
PacienteAnos                     int64
PacienteSexo                    object
DiasInternacion                  int64
CondicionAlCierre               object
NotaFechaHora                   object
ArqutipoNombre                  object
NotaArquetipoValorIngresado    float64
dtype: object

In [5]:
#ID de int a object
df['ID'] = df['ID'].astype(object)

#PacienteSexo y CondicionAlCierre de object a category
df['PacienteSexo'] = df['PacienteSexo'].astype('category') 
df['CondicionAlCierre'] = df['CondicionAlCierre'].astype('category') 

#FechaIngreso, PacienteFechaNacimiento y NotaFechaHora de object a datatime
df['FechaIngreso'] = pd.to_datetime(df['FechaIngreso'], format='%Y-%m-%d')
df['PacienteFechaNacimiento'] = pd.to_datetime(df['PacienteFechaNacimiento'], format='%Y-%m-%d')
df['NotaFechaHora'] = pd.to_datetime(df['NotaFechaHora'], format='%Y-%m-%d %H:%M')

#DiasInternacion que sea medido como días
pd.to_timedelta(df['DiasInternacion'], unit='D')


df.dtypes

ID                                     object
FechaIngreso                   datetime64[ns]
PacienteFechaNacimiento        datetime64[ns]
PacienteAnos                            int64
PacienteSexo                         category
DiasInternacion                         int64
CondicionAlCierre                    category
NotaFechaHora                  datetime64[ns]
ArqutipoNombre                         object
NotaArquetipoValorIngresado           float64
dtype: object

Se crea una columna que diga cuántos días de internación llevaba el paciente al momento en el que se le realizó la medición del signo vital

In [6]:
x = (df['NotaFechaHora'] - df['FechaIngreso']) / np.timedelta64(1, 'D')

df.insert(8, 'NotaDia', x)

df['NotaDia'] = df['NotaDia'].astype(int)

df.head()

,ID,FechaIngreso,PacienteFechaNacimiento,PacienteAnos,PacienteSexo,DiasInternacion,CondicionAlCierre,NotaFechaHora,NotaDia,ArqutipoNombre,NotaArquetipoValorIngresado
0,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,0,Frecuencia Cardiaca,78.0
1,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,0,Frecuencia respiratoria,18.0
2,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,0,Presion Arterial Maxima,120.0
3,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,0,Presion Arterial Minima,80.0
4,318849,2022-01-01,1990-08-10,31,FEMENINO,2,ALTA MEDICA,2022-01-01 09:19:28,0,Temperatura,36.3


In [7]:
df['NotaDia'].value_counts()

1      91832
0      73967
2      61714
3      48342
4      38708
       ...  
129       18
130       18
134       18
131       12
135        7
Name: NotaDia, Length: 136, dtype: int64

# Recorte de datos

se crea un nuevo data set solo con los pacientes que hayan estado internado más de dos días. De ellos solo quedan en el nuevo DS las mediciones que se le haya hecho en sus dosm primeros días de internación

In [8]:
#DF de casos con mas de dos días de internacion extrayendo los valores de sus dos primeros dias
dfCorto = df.loc[(df['DiasInternacion'] > 2 ) & (df['NotaDia'] < 2)]
dfCorto.reset_index(inplace=True)
dfCorto.drop(['index'], axis=1, inplace=True)

dfCorto.head()

,ID,FechaIngreso,PacienteFechaNacimiento,PacienteAnos,PacienteSexo,DiasInternacion,CondicionAlCierre,NotaFechaHora,NotaDia,ArqutipoNombre,NotaArquetipoValorIngresado
0,318853,2022-01-01,1959-02-15,62,MASCULINO,109,DEFUNCION,2022-01-01 10:46:38,0,Frecuencia Cardiaca,80.0
1,318853,2022-01-01,1959-02-15,62,MASCULINO,109,DEFUNCION,2022-01-01 10:46:38,0,Frecuencia respiratoria,20.0
2,318853,2022-01-01,1959-02-15,62,MASCULINO,109,DEFUNCION,2022-01-01 10:46:38,0,Presion Arterial Maxima,120.0
3,318853,2022-01-01,1959-02-15,62,MASCULINO,109,DEFUNCION,2022-01-01 10:46:38,0,Presion Arterial Minima,80.0
4,318853,2022-01-01,1959-02-15,62,MASCULINO,109,DEFUNCION,2022-01-01 10:46:38,0,Presion Arterial Media,93.0


In [9]:
dfCorto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96797 entries, 0 to 96796
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           96797 non-null  object        
 1   FechaIngreso                 96797 non-null  datetime64[ns]
 2   PacienteFechaNacimiento      96797 non-null  datetime64[ns]
 3   PacienteAnos                 96797 non-null  int64         
 4   PacienteSexo                 96797 non-null  category      
 5   DiasInternacion              96797 non-null  int64         
 6   CondicionAlCierre            96797 non-null  category      
 7   NotaFechaHora                96797 non-null  datetime64[ns]
 8   NotaDia                      96797 non-null  int64         
 9   ArqutipoNombre               96797 non-null  object        
 10  NotaArquetipoValorIngresado  96796 non-null  float64       
dtypes: category(2), datetime64[ns](3), float6

In [10]:
dfCorto.nunique()

ID                              2047
FechaIngreso                     307
PacienteFechaNacimiento         1775
PacienteAnos                      96
PacienteSexo                       2
DiasInternacion                   54
CondicionAlCierre                  6
NotaFechaHora                  12355
NotaDia                            2
ArqutipoNombre                     7
NotaArquetipoValorIngresado      282
dtype: int64

# Pivoteo del DS original al que se va a crear

Se crea la estructura de nuevo DS que tendrá los datos que son únicos de cada paciente y que en el DS original se repiten en cada registro: ID, edad (se descarta fecha de nacimiento por innecesario), sexo, días de internación y condición al cierre (aunque este último data fnalmente no se usará)

In [11]:
#Df con una línea por paciente
dfDEF = pd.DataFrame(dfCorto[['ID', 'PacienteAnos', 'PacienteSexo','DiasInternacion', 'CondicionAlCierre']])

dfDEF = dfDEF.drop_duplicates()
dfDEF.reset_index(inplace=True)
dfDEF.drop(['index'], axis=1, inplace=True)


dfDEF.head()

,ID,PacienteAnos,PacienteSexo,DiasInternacion,CondicionAlCierre
0,318853,62,MASCULINO,109,DEFUNCION
1,318906,57,FEMENINO,15,ALTA MEDICA
2,318909,50,MASCULINO,9,ALTA MEDICA
3,318942,44,MASCULINO,3,ALTA MEDICA
4,318970,66,MASCULINO,6,ALTA MEDICA


In [12]:
dfDEF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 0 to 2046
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ID                 2047 non-null   object  
 1   PacienteAnos       2047 non-null   int64   
 2   PacienteSexo       2047 non-null   category
 3   DiasInternacion    2047 non-null   int64   
 4   CondicionAlCierre  2047 non-null   category
dtypes: category(2), int64(2), object(1)
memory usage: 52.6+ KB


Tras un análisis de los datos existentes, los objetivos de aplicar un modelo predictivo y las mediciones que se consideran más relevantes dentro del recorte temporal ya seleccionado, se extraen los datos de la primera mediación de cada paciente para cada uno de los signos vitales y se colocan en una columna. Luego se agrgan tres columnas más por cada signo vital con las tres primeras mediciones del segundo día de internación.  

In [13]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Frecuencia Cardiaca') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Frecuencia Cardiaca') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fc1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('fc1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('fc1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fc2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('fc2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fc2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('fc2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fc2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('fc2C')]= np.nan



In [14]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Frecuencia respiratoria') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Frecuencia respiratoria') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fr1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('fr1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('fr1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fr2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('fr2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fr2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('fr2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('fr2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('fr2C')]= np.nan

In [15]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Presion Arterial Maxima') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Presion Arterial Maxima') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMAX1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('PAMAX1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('PAMAX1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMAX2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMAX2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMAX2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMAX2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMAX2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMAX2C')]= np.nan

In [16]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Presion Arterial Minima') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Presion Arterial Minima') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMIN1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('PAMIN1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('PAMIN1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMIN2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMIN2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMIN2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMIN2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMIN2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMIN2C')]= np.nan

In [17]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Presion Arterial Media') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Presion Arterial Media') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('PAMEDIA1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('PAMEDIA1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('PAMEDIA2C')]= np.nan

In [18]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Temperatura') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Temperatura') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('TEMP1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('TEMP1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('TEMP1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('TEMP2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('TEMP2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('TEMP2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('TEMP2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('TEMP2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('TEMP2C')]= np.nan

In [19]:
for i in dfCorto['ID'].unique():
        nota1 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Saturacion O2') & (dfCorto['NotaDia'] == 0)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        nota2 = dfCorto[(dfCorto['ID'] == i) & (dfCorto['ArqutipoNombre']== 'Saturacion O2') & (dfCorto['NotaDia'] == 1)]\
        ['NotaArquetipoValorIngresado'].to_numpy()
        
        #1º toma
        try: 
            dfDEF.loc[(dfDEF.ID == i),('O2SAT1')]= nota1[0]
        except:
            try:
                dfDEF.loc[(dfDEF.ID == i),('O2SAT1')]= nota2[0]            
            except:
                dfDEF.loc[(dfDEF.ID == i),('O2SAT1')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('O2SAT2A')]= nota2[0]
        except:
            dfDEF.loc[(dfDEF.ID == i),('O2SAT2A')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('O2SAT2B')]= nota2[1]
        except:
            dfDEF.loc[(dfDEF.ID == i),('O2SAT2B')]= np.nan
        
        try: 
            dfDEF.loc[(dfDEF.ID == i),('O2SAT2C')]= nota2[2]
        except:
            dfDEF.loc[(dfDEF.ID == i),('O2SAT2C')]= np.nan

In [20]:
dfDEF.columns

Index(['ID', 'PacienteAnos', 'PacienteSexo', 'DiasInternacion',
       'CondicionAlCierre', 'fc1', 'fc2A', 'fc2B', 'fc2C', 'fr1', 'fr2A',
       'fr2B', 'fr2C', 'PAMAX1', 'PAMAX2A', 'PAMAX2B', 'PAMAX2C', 'PAMIN1',
       'PAMIN2A', 'PAMIN2B', 'PAMIN2C', 'PAMEDIA1', 'PAMEDIA2A', 'PAMEDIA2B',
       'PAMEDIA2C', 'TEMP1', 'TEMP2A', 'TEMP2B', 'TEMP2C', 'O2SAT1', 'O2SAT2A',
       'O2SAT2B', 'O2SAT2C'],
      dtype='object')

In [21]:
dfDEF.isnull().sum()

ID                      0
PacienteAnos            0
PacienteSexo            0
DiasInternacion         0
CondicionAlCierre       0
fc1                     2
fc2A                  102
fc2B                  147
fc2C                  412
fr1                     3
fr2A                  102
fr2B                  149
fr2C                  414
PAMAX1                212
PAMAX2A               287
PAMAX2B               338
PAMAX2C               556
PAMIN1                212
PAMIN2A               287
PAMIN2B               338
PAMIN2C               562
PAMEDIA1              778
PAMEDIA2A            1223
PAMEDIA2B            1327
PAMEDIA2C            1370
TEMP1                   3
TEMP2A                104
TEMP2B                167
TEMP2C                480
O2SAT1                  6
O2SAT2A               105
O2SAT2B               175
O2SAT2C               558
dtype: int64

Por la gran cantidad de pacientes que no se le ha medido Presión Arterial Media (699 de 2047 pacientes) y en base a que es importante pero no determinante como parámetro médico (al ser solo una media entra la Presión Alta y la Baja), se borra ese parámetro para no distoricionar el resultado final.

In [22]:
dfDEF.drop(['PAMEDIA1','PAMEDIA2A', 'PAMEDIA2B', 'PAMEDIA2C'], axis=1, inplace=True)

Se elimina todo paciente que no tenga al menos una medición en alguno de los parámetros medidos

In [23]:
dfDEF.drop(dfDEF[(dfDEF['fc1'].isnull()) | (dfDEF['fr1'].isnull()) | (dfDEF['PAMAX1'].isnull()) | 
                 (dfDEF['PAMIN1'].isnull()) | (dfDEF['TEMP1'].isnull()) | (dfDEF['O2SAT1'].isnull()) | 
                 (dfDEF['fr1'].isnull())].index, inplace = True)
dfDEF

,ID,PacienteAnos,PacienteSexo,DiasInternacion,CondicionAlCierre,fc1,fc2A,fc2B,fc2C,fr1,...,PAMIN2B,PAMIN2C,TEMP1,TEMP2A,TEMP2B,TEMP2C,O2SAT1,O2SAT2A,O2SAT2B,O2SAT2C
0,318853,62,MASCULINO,109,DEFUNCION,80.0,93.0,89.0,90.0,20.0,...,70.0,70.0,36.2,36.5,36.4,NaN,97.0,94.0,95.0,92.0
1,318906,57,FEMENINO,15,ALTA MEDICA,75.0,91.0,85.0,86.0,19.0,...,76.0,85.0,36.4,36.0,36.6,36.0,90.0,99.0,99.0,96.0
2,318909,50,MASCULINO,9,ALTA MEDICA,80.0,61.0,72.0,69.0,20.0,...,71.0,83.0,36.7,36.0,36.0,36.0,89.0,96.0,99.0,97.0
3,318942,44,MASCULINO,3,ALTA MEDICA,85.0,63.0,74.0,70.0,20.0,...,80.0,60.0,36.8,36.0,36.3,36.0,96.0,97.0,97.0,96.0
4,318970,66,MASCULINO,6,ALTA MEDICA,89.0,83.0,76.0,87.0,20.0,...,60.0,80.0,35.8,36.0,36.0,36.0,95.0,97.0,97.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,386408,36,FEMENINO,3,ALTA MEDICA,78.0,81.0,80.0,NaN,20.0,...,80.0,NaN,36.0,36.5,35.5,NaN,98.0,98.0,98.0,NaN
2043,386436,24,FEMENINO,5,ALTA MEDICA,78.0,110.0,97.0,83.0,22.0,...,77.0,81.0,36.0,38.0,37.5,37.3,100.0,96.0,96.0,96.0
2044,386863,42,MASCULINO,3,DEFUNCION,105.0,94.0,98.0,100.0,20.0,...,110.0,103.0,36.2,36.0,36.1,36.1,98.0,98.0,98.0,97.0
2045,386947,30,MASCULINO,4,ALTA MEDICA,80.0,80.0,75.0,81.0,20.0,...,60.0,80.0,36.0,36.0,36.0,36.0,96.0,96.0,97.0,NaN


Al borrar a quienes no tenían medida nunca algún parámetro quedaron 1831 de 2047 (se borraron: 216)

In [24]:
dfDEF.isnull().sum()
#df['ID'] = df['ID'].astype(object)


ID                     0
PacienteAnos           0
PacienteSexo           0
DiasInternacion        0
CondicionAlCierre      0
fc1                    0
fc2A                  34
fc2B                  62
fc2C                 285
fr1                    0
fr2A                  34
fr2B                  64
fr2C                 287
PAMAX1                 0
PAMAX2A               74
PAMAX2B              123
PAMAX2C              341
PAMIN1                 0
PAMIN2A               74
PAMIN2B              123
PAMIN2C              347
TEMP1                  0
TEMP2A                36
TEMP2B                80
TEMP2C               351
O2SAT1                 0
O2SAT2A               35
O2SAT2B               87
O2SAT2C              417
dtype: int64

Para las mediciones que faltan se rellena con la medición inmediatamente anterior, naturalmente del mismo paciente y del mismo parámetro 

In [25]:
dfDEF = dfDEF.fillna(method = 'ffill', axis=1)

dfDEF.isnull().sum()

ID                   0
PacienteAnos         0
PacienteSexo         0
DiasInternacion      0
CondicionAlCierre    0
fc1                  0
fc2A                 0
fc2B                 0
fc2C                 0
fr1                  0
fr2A                 0
fr2B                 0
fr2C                 0
PAMAX1               0
PAMAX2A              0
PAMAX2B              0
PAMAX2C              0
PAMIN1               0
PAMIN2A              0
PAMIN2B              0
PAMIN2C              0
TEMP1                0
TEMP2A               0
TEMP2B               0
TEMP2C               0
O2SAT1               0
O2SAT2A              0
O2SAT2B              0
O2SAT2C              0
dtype: int64

Se guarda el data set recortado en un nuevo archivo excel y en uno de tipo CSV

In [26]:
dfDEF.to_excel('dfRECORTADO.xlsx', index = False)

In [27]:
dfDEF.to_csv('dfRECORTADO.csv', index = False)

# Conclusiones

En base a los datos existentes y las posibilidades del modelo predictivo se decide realizar un recorte de la información original. 
Del primer Data Set se extraen solo los pacientes que hayan estado más de dos días internados, y de los mismos se recogen las mediciones de sus dos primeras jornadas dentro de la Terapia Intensiva.
Por último, se definen otros aspectos para terminar de crear el Data Set que se empleará para el modelo de clasificación: eliminar a los pacientes que no tengan ninguna medición en alguno de los signos vitales analizados y rellenar los datos faltantes de alguna medición con los resultados que obtuvo el paciente en la medición inmediatamente anterior.
El nuevo Data Set queda entonces con 1831 filas (una para cada paciente) y 29 columnas. En las columnas, 5 responden a los datos personales del paciente y las otras 24 a las mediciones de sus signos vitales. Dentro de esas 24 hay 4 columnas para cada uno de los 6 signos vitales medidos. Las 4 columnas son una para la primera medición y las otras 3 para las 3 primeras medición del segundo día de internación.
El Data Set original no ha sido modificado, pudiéndose entonces utilizar la gran cantidad de información que se ha descartado para este proyecto en otros análisis.